In [ ]:
# import os
# import pandas as pd
# import glob

# def merge_mql_with_monthly_data(base_path):
#     """
#     Merge MQLs with monthly activity data for all available months.
    
#     Args:
#     base_path (str): Base directory path containing the data files
    
#     Returns:
#     dict: A dictionary of merged DataFrames for each month
#     """
#     # Create a merged_mql folder if it doesn't exist
#     merged_folder = os.path.join(base_path, 'merged_mql')
#     os.makedirs(merged_folder, exist_ok=True)
    
#     # Read MQLs data
#     mql_df = pd.read_csv(os.path.join(base_path, 'MQLs.csv'))
    
#     # Convert 'Most Recent MQL Date' to datetime
#     mql_df['Most Recent MQL Date'] = pd.to_datetime(mql_df['Most Recent MQL Date'])
    
#     # Find all monthly CSV files (excluding MQLs.csv and other potential CSVs)
#     monthly_files = glob.glob(os.path.join(base_path, '*-*.csv'))
#     monthly_files = [f for f in monthly_files if f != os.path.join(base_path, 'MQLs.csv')]
    
#     # Dictionary to store merged results
#     merged_results = {}
    
#     # Comprehensive month mapping
#     month_map = {
#         'jan': 1, 'january': 1,
#         'feb': 2, 'february': 2,
#         'mar': 3, 'march': 3,
#         'apr': 4, 'april': 4,
#         'may': 5,
#         'jun': 6, 'june': 6,
#         'jul': 7, 'july': 7,
#         'aug': 8, 'august': 8,
#         'sep': 9, 'september': 9,
#         'oct': 10, 'october': 10,
#         'nov': 11, 'november': 11,
#         'dec': 12, 'december': 12
#     }
    
#     # Process each monthly file
#     for monthly_file in monthly_files:
#         # Extract month from filename
#         filename = os.path.basename(monthly_file)
        
#         # Skip if already processed or not a typical month file
#         if filename.startswith('merged_mql_') or 'Pivot' in filename:
#             continue
        
#         # Split filename to get month part
#         month_part = filename.split('-')[0].lower()
        
#         # Get month number
#         month_num = month_map.get(month_part)
        
#         if month_num is None:
#             print(f"Warning: Could not parse month from {filename}")
#             continue
        
#         # Filter MQLs for the specific month
#         monthly_mql = mql_df[mql_df['Most Recent MQL Date'].dt.month == month_num]
        
#         # Remove duplicate MQLs, keeping the first occurrence
#         monthly_mql = monthly_mql.drop_duplicates(subset='Email', keep='first')
        
#         # Read monthly activity data with mixed type handling
#         monthly_activity_df = pd.read_csv(
#             monthly_file, 
#             low_memory=False,  # Handle mixed types
#             dtype={  # Specify critical columns if needed
#                 'Email Address': str
#             }
#         )
        
#         # Merge MQLs with monthly activity data
#         merged_df = pd.merge(
#             monthly_activity_df, 
#             monthly_mql, 
#             left_on='Email Address', 
#             right_on='Email', 
#             how='left'
#         )
        
#         # Remove any duplicate rows
#         merged_df = merged_df.drop_duplicates()
        
#         # Save merged data to CSV in the merged_mql folder
#         output_filename = os.path.join(merged_folder, f'merged_mql_{filename}')
#         merged_df.to_csv(output_filename, index=False)
        
#         # Store in results dictionary
#         merged_results[filename] = merged_df
        
#         print(f"Processed {filename}: {merged_df.shape[0]} rows, {merged_df.shape[1]} columns")
    
#     return merged_results

# # Set the base path to the current directory
# base_path = r'C:\Users\itsoh\Downloads\TE\TE'

# # Run the merger
# results = merge_mql_with_monthly_data(base_path)

# # Print out the location of merged files
# print(f"\nMerged files saved in: {os.path.join(base_path, 'merged_mql')}")

Processed apr-24.csv: 13599 rows, 80 columns
Processed Aug-24.csv: 11389 rows, 80 columns
Processed dec-23.csv: 15590 rows, 80 columns
Processed feb-24.csv: 19151 rows, 80 columns
Processed jan-24.csv: 16038 rows, 80 columns
Processed Jul-24.csv: 8513 rows, 80 columns
Processed June-24.csv: 12381 rows, 80 columns
Processed mar-24.csv: 13331 rows, 80 columns
Processed May-24.csv: 12944 rows, 80 columns
Processed Nov-23.csv: 14271 rows, 80 columns
Processed Oct-23.csv: 20144 rows, 80 columns
Processed Sep-24.csv: 7964 rows, 80 columns

Merged files saved in: C:\Users\itsoh\Downloads\TE\TE\merged_mql


In [2]:
import os
import pandas as pd
from sklearn.preprocessing import LabelEncoder

def encode_emails_by_month(base_path):
    """
    Encode emails separately for each month while preserving all original data
    
    Args:
    base_path (str): Path to the merged_mql folder
    
    Returns:
    pd.DataFrame: Combined DataFrame with monthly email encodings and full data
    """
    # Find all merged MQL files
    merged_mql_folder = os.path.join(base_path, 'merged_mql')
    monthly_files = [f for f in os.listdir(merged_mql_folder) if f.startswith('merged_mql_') and f.endswith('.csv')]
    
    # List to store processed dataframes
    processed_dfs = []
    
    # Process each monthly file
    for filename in monthly_files:
        # Read the file
        file_path = os.path.join(merged_mql_folder, filename)
        df = pd.read_csv(file_path, low_memory=False)
        
        # Extract month from filename
        month = filename.replace('merged_mql_', '').replace('.csv', '')
        
        # Create a label encoder for this month's emails
        email_encoder = LabelEncoder()
        
        # Encode emails for this month
        df['Monthly_Encoded_Email'] = email_encoder.fit_transform(df['Email Address'])
        
        # Add month column
        df['Month'] = month
        
        processed_dfs.append(df)
        
        print(f"Processed {filename}: {len(df)} rows")
    
    # Combine all processed dataframes
    combined_df = pd.concat(processed_dfs, ignore_index=True)
    
    # Save combined dataset
    output_path = os.path.join(base_path, 'monthly_encoded_full_data.csv')
    combined_df.to_csv(output_path, index=False)
    
    print(f"\nCombined monthly encoded data saved to {output_path}")
    print(f"Total rows: {len(combined_df)}")
    print(f"Total columns: {len(combined_df.columns)}")
    
    return combined_df

# Set the base path
base_path = r'C:\Users\itsoh\Downloads\TE\TE'

# Run the processing
monthly_encoded_full_data = encode_emails_by_month(base_path)

Processed merged_mql_apr-24.csv: 13599 rows
Processed merged_mql_Aug-24.csv: 11389 rows
Processed merged_mql_dec-23.csv: 15590 rows
Processed merged_mql_feb-24.csv: 19151 rows
Processed merged_mql_jan-24.csv: 16038 rows
Processed merged_mql_Jul-24.csv: 8513 rows
Processed merged_mql_June-24.csv: 12381 rows
Processed merged_mql_mar-24.csv: 13331 rows
Processed merged_mql_May-24.csv: 12944 rows
Processed merged_mql_Nov-23.csv: 14271 rows
Processed merged_mql_Oct-23.csv: 20144 rows
Processed merged_mql_Sep-24.csv: 7964 rows

Combined monthly encoded data saved to C:\Users\itsoh\Downloads\TE\TE\monthly_encoded_full_data.csv
Total rows: 165315
Total columns: 82
